In [0]:
import numpy as np
import pandas as pd
import zipfile
# !pip install fastkml
# from fastkml import kml
from xml.dom import minidom  # to read KML
import folium  # create map
from math import atan2, cos, sin, sqrt, radians

## Scraping results

In [0]:
url = "http://vacationwithoutacar.com/PDF/INTERIM_GVRAT_Tracking_Sheet.html"
dfs = pd.read_html(url)

In [59]:
czechs = dfs[0][dfs[0].E == "CZ"].copy()
czechs

,Unnamed: 0,A,B,C,D,E,F,G,H,I,M,N,O,P,Q,R
3370,3371,3367,12541,Richard Bijecek,GVRAT,CZ,M,37,26.4,Memphis,4.2%,08/04/2020,6,12,8,0
3574,3575,3571,2225,Karla Fejfarova,GVRAT,CZ,F,40,26.0,Memphis,4.1%,08/05/2020,10,6,10,0
5085,5086,5082,4277,Pavlina Polaskova,GVRAT,CZ,F,38,22.5,approaching Mississippi River,3.5%,08/20/2020,4,5,13,0
7134,7135,7131,4370,Petr Simecek,GVRAT,CZ,M,40,19.1,West Memphis AR,3.0%,09/09/2020,7,6,6,0


In [60]:
extra_info = pd.DataFrame.from_dict({'C': ['Richard Bijecek', 'Karla Fejfarova', 'Pavlina Polaskova', 'Petr Simecek'],
                                     'icon': ['male', 'female', 'female', 'male'],
                                     'color': ['darkgreen', 'orange', 'pink', 'darkpurple']})
czechs = czechs.merge(extra_info, how="left", on="C")
czechs

,Unnamed: 0,A,B,C,D,E,F,G,H,I,M,N,O,P,Q,R,icon,color
0,3371,3367,12541,Richard Bijecek,GVRAT,CZ,M,37,26.4,Memphis,4.2%,08/04/2020,6,12,8,0,male,darkgreen
1,3575,3571,2225,Karla Fejfarova,GVRAT,CZ,F,40,26.0,Memphis,4.1%,08/05/2020,10,6,10,0,female,orange
2,5086,5082,4277,Pavlina Polaskova,GVRAT,CZ,F,38,22.5,approaching Mississippi River,3.5%,08/20/2020,4,5,13,0,female,pink
3,7135,7131,4370,Petr Simecek,GVRAT,CZ,M,40,19.1,West Memphis AR,3.0%,09/09/2020,7,6,6,0,male,darkpurple


## Calculating position

In [0]:
archive = zipfile.ZipFile('GVRAT Course.kml.kmz', 'r')
kmlfile = archive.open('doc.kml').read()

In [0]:
def calc_distance(origin, destination):
        """great-circle distance between two points on a sphere
           from their longitudes and latitudes"""
        lat1, lon1 = origin
        lat2, lon2 = destination
        radius = 3965  # The radius of Earth in miles

        dlat = radians(lat2-lat1)
        dlon = radians(lon2-lon1)
        a = (sin(dlat/2) * sin(dlat/2) + cos(radians(lat1)) * cos(radians(lat2)) *
             sin(dlon/2) * sin(dlon/2))
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        d = radius * c

        return d

In [0]:
# code using fastkml

# k = kml.KML()
# k.from_string(kmlfile)
# len(k._features)

# curve = k._features[0]._features[0]
# len(curve.geometry.coords)

# curve.geometry.coords[0]
# be careful - 3 coordinates

# calc_distance(curve.geometry.coords[0], curve.geometry.coords[0])

In [0]:
xmldoc = minidom.parse(archive.open('doc.kml'))
kml = xmldoc.getElementsByTagName("kml")[0]
document = kml.getElementsByTagName("Document")[0]
placemarks = document.getElementsByTagName("Placemark")
linestring = document.getElementsByTagName("LineString")

In [8]:
nodes = []
for placemark in placemarks:
  nodename = placemark.getElementsByTagName("name")[0].firstChild.data[:-1]
  coords = placemark.getElementsByTagName("coordinates")[0].firstChild.data
  lst1 = coords.split(",")
  longitude = float(lst1[0])
  latitude = float(lst1[1])
  nodes.append((latitude, longitude))
len(nodes)

1

In [0]:
coords = linestring[0].getElementsByTagName("coordinates")[0].firstChild.data

In [10]:
tmp = [x.split(",") for x in coords.split("\n          ")[1:]]
tmp[:10]

[['-90.310675', '34.995691', '0'],
 ['-90.310941', '34.995653', '0'],
 ['-90.31106', '34.995639', '0'],
 ['-90.31119', '34.995599', '0'],
 ['-90.311406', '34.995616', '0'],
 ['-90.311409', '34.995616', '0'],
 ['-90.31143', '34.995621', '0'],
 ['-90.312308', '34.995562', '0'],
 ['-90.312909', '34.995467', '0'],
 ['-90.313752', '34.995245', '0']]

In [11]:
lat_long = [(float(lat), float(long)) for long, lat, _ in tmp]
lat_long[:10]

[(34.995691, -90.310675),
 (34.995653, -90.310941),
 (34.995639, -90.31106),
 (34.995599, -90.31119),
 (34.995616, -90.311406),
 (34.995616, -90.311409),
 (34.995621, -90.31143),
 (34.995562, -90.312308),
 (34.995467, -90.312909),
 (34.995245, -90.313752)]

In [12]:
calc_distance(lat_long[0], lat_long[1])

0.015307169857920968

In [13]:
lat_long_dist = np.array([calc_distance(lat_long[i], lat_long[i+1]) for i in range(len(lat_long) - 1)])
lat_long_dist_cumsum = np.cumsum(lat_long_dist)
sum(lat_long_dist)

634.8406921492539

In [14]:
czechs.H.astype("float")

3370    26.4
3574    26.0
5085    22.5
7134    19.1
Name: H, dtype: float64

In [15]:
b = np.where(lat_long_dist_cumsum >= 20)[0].min()
b

294

In [0]:
def interpolate_points(point1, point2, dist1, dist2, dist):
  alpha = (dist - dist1) / (dist2 - dist1)
  lat = point1[0] + (point2[0] - point1[0]) * alpha
  long = point1[1] + (point2[1] - point1[1]) * alpha
  return lat, long


In [17]:
interpolate_points(lat_long[b], lat_long[b+1], lat_long_dist_cumsum[b-1], lat_long_dist_cumsum[b], 20)

(35.140957001355694, -90.13423086780155)

In [0]:
def find_position_after_k_miles(k):
  k = float(k)
  bp = np.where(lat_long_dist_cumsum >= k)[0].min()
  return interpolate_points(lat_long[bp], lat_long[bp+1], lat_long_dist_cumsum[bp-1], lat_long_dist_cumsum[bp], k)

In [56]:
czechs['position'] = czechs.H.apply(find_position_after_k_miles)
czechs

,Unnamed: 0,A,B,C,D,E,F,G,H,I,M,N,O,P,Q,R,position
3370,3371,3367,12541,Richard Bijecek,GVRAT,CZ,M,37,26.4,Memphis,4.2%,08/04/2020,6,12,8,0,"(35.12455321220573, -90.05478044800826)"
3574,3575,3571,2225,Karla Fejfarova,GVRAT,CZ,F,40,26.0,Memphis,4.1%,08/05/2020,10,6,10,0,"(35.12456386082135, -90.06182294084235)"
5085,5086,5082,4277,Pavlina Polaskova,GVRAT,CZ,F,38,22.5,approaching Mississippi River,3.5%,08/20/2020,4,5,13,0,"(35.13961676898447, -90.09615851372362)"
7134,7135,7131,4370,Petr Simecek,GVRAT,CZ,M,40,19.1,West Memphis AR,3.0%,09/09/2020,7,6,6,0,"(35.13791056011299, -90.1496917518698)"


## Map

In [20]:
lat_range = min([lat for lat, _ in lat_long]), max([lat for lat, _ in lat_long])
long_range = min([long for _, long in lat_long]), max([long for _, long in lat_long])
lat_range, long_range

((34.9748, 36.70243), (-90.32706, -81.646902))

In [0]:
lat_center = czechs['position'].apply(lambda x: x[0]).mean()
long_center = czechs['position'].apply(lambda x: x[1]).mean()

In [58]:
m = folium.Map(
    width='100%', 
    height='100%',
    location=[lat_center, long_center],
    zoom_start=12
)

folium.PolyLine(lat_long).add_to(m)

for i in range(czechs.shape[0]):
  folium.Marker(czechs.position.iloc[i], tooltip=czechs.C.iloc[i], 
                icon=folium.Icon(icon=czechs.icon.iloc[i], color=czechs.color.iloc[i], prefix="fa"),
                popup="{},\n{}mi,\n{}".format(czechs.C.iloc[i], czechs.H.iloc[i], czechs.position.iloc[i])).add_to(m)

m

In [0]:
m.save(outfile= "test.html")

In [33]:
help(folium.Icon)

Help on class Icon in module folium.map:

class Icon(branca.element.MacroElement)
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the `prefix` as well.
 |  angle : int, default 0
 |      The icon will be rotated by this amount of degrees.
 